In [2]:
import pandas as pd
import json
import ast

In [31]:
import pandas as pd
import json
import ast

class Append:
    def __init__(self, df2):
        self.df = df2
        self.df_result = pd.DataFrame()

    def separation(self):
        for i in range(len(self.df)):
            df2t = self.df.iloc[[i]]  # Process each row
            df2t = pd.DataFrame(df2t)
            df2t['data'] = df2t['neo_data']
            df2t = df2t.drop(columns=['neo_data'])
            df2t = df2t.astype(str)

            parts = df2t['data'].str.split(',')
            parts1 = df2t['data'].str.split('[')
            approachdata = parts1.str[1]
            parts2 = approachdata.str.split(']')
            approach = parts2.str[0]
            orbit = parts2.str[1]
            parts3 = orbit.str.split(',')
            sentry = parts3.str[26]
            orbit = parts3.str[0:26]

            df3 = pd.DataFrame(approach)
            df3.rename(columns={'data': 'close_approach_data'}, inplace=True)

            df4 = pd.DataFrame(orbit)
            df4.rename(columns={'data': 'orbit_data'}, inplace=True)

            df3 = pd.concat([df3, df4], axis=1)

            df5 = pd.DataFrame(sentry)
            df5.rename(columns={'data': 'sentry_object_data'}, inplace=True)

            df3 = pd.concat([df3, df5], axis=1)

            df2t['links'] = parts.str[0]
            df2t['id'] = parts.str[1]
            df2t['neo_reference_id'] = parts.str[2]
            df2t['name'] = parts.str[3]
            df2t['designation'] = parts.str[4]
            df2t['nasa_jpl_url'] = parts.str[5]
            df2t['absolute_magnitude_h'] = parts.str[6]
            df2t['estimated_diameter'] = parts.str[7]+parts.str[8]+parts.str[9]+parts.str[10]+parts.str[11]+parts.str[12]+parts.str[13]+parts.str[14]
            df2t['estimated_diameter'] = df2t['estimated_diameter'].apply(lambda x: '{' + x)
            df2t['is_potentially_hazardous_asteroid'] = parts.str[15]

            df2t = pd.concat([df2t, df3], axis=1)

            self.df_result = pd.concat([self.df_result, df2t], ignore_index=True)

        self.df_result = self.df_result.drop(columns=['data'])
        self.df_result.reset_index(drop=True, inplace=True)

        return self.df_result

class Clean(Append):
    def __init__(self, df_result):
        super().__init__(df_result)
        self.df_result = self.separation()
        self.dffinal = pd.DataFrame()

    def clean_json_links(self, row):
        corrected_json = row.replace("'", '"') + '}'  # Assuming you want to replace single quotes with double quotes
        return {'links': corrected_json}


    def clean_json_diameter(self, row):
        corrected_json = row.replace("'", '"') + '}'  # Assuming you want to replace single quotes with double quotes
        return {'estimated_diameter': corrected_json}

    def manipulate_json(self):

        for i in range(len(self.df_result)):
            row = self.df_result['links'].iloc[i]
            row = self.clean_json_links(row)  # Use the class method

            try:
                # Load JSON from the 'links' key
                json_dict = json.loads(row['links'])
                # Extract 'self' key from the JSON dictionary
                new_data_links = {'self': json_dict['links']['self']}
                # Append new data as a DataFrame row
                self.dffinal = pd.concat([self.dffinal, pd.DataFrame([new_data_links])], ignore_index=True)
            except KeyError as e:
                print(f"Key error: {e} - Check that the JSON data and the key 'self' exist.")
            except TypeError as e:
                print(f"Type error: {e} - Input should be a dictionary with a 'links' key containing a JSON string.")
            except json.JSONDecodeError as e:
                print(f"JSON decode error: {e} - Check the JSON format.")


        for i in range(len(self.df_result)):
            row = self.df_result['estimated_diameter'].iloc[i]
            row = self.clean_json_diameter(row)  # Use the class method

            try:
                # Load JSON from the 'links' key
                json_dict = json.loads(row['estimated_diameter'])
                # Extract 'self' key from the JSON dictionary
                new_data_diameter = {'estimated_diameter': json_dict['estimated_diameter']['kilometers']}
                # Append new data as a DataFrame row
                self.dffinal = pd.concat([self.dffinal, pd.DataFrame([new_data_diameter])], ignore_index=True)
            except KeyError as e:
                print(f"Key error: {e} - Check that the JSON data and the key 'estimated_diameter' exist.")
            except TypeError as e:
                print(f"Type error: {e} - Input should be a dictionary with a 'estimated_diameter' key containing a JSON string.")
            except json.JSONDecodeError as e:
                print(f"JSON decode error: {e} - Check the JSON format.")
            

        return self.dffinal

    def merge_json(self):
        test1 = self.manipulate_json()
        test1.rename(columns={'self': 'links'}, inplace=True)
        df = self.df_result
        df = df.drop('links', axis=1)
        self.dfmerge = pd.merge(test1, df, left_index=True, right_index=True)
        return self.dfmerge
    

    def table(self):
        self.dfmerge = self.merge_json()
        self.dfmerge['id'] = self.dfmerge['id'].str.replace('\'', '')
        self.dfmerge['id'] = self.dfmerge['id'].str.replace('{', '')
        self.dfmerge['id'] = self.dfmerge['id'].str.replace('}', '')
        self.dfmerge['id'] = self.dfmerge['id'].str.replace(']', '')
        self.dfmerge['id'] = self.dfmerge['id'].str.replace('[', '')
        parts = self.dfmerge['id'].str.split(':')
        self.dfmerge['id'] = parts.str[1]
        self.dfmerge['id'] = self.dfmerge['id'].astype(int)

        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].str.replace('\'', '')
        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].str.replace('{', '')
        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].str.replace('}', '')
        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].str.replace(']', '')
        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].str.replace('[', '')
        parts = self.dfmerge['neo_reference_id'].str.split(':')
        self.dfmerge['neo_reference_id'] = parts.str[1]
        self.dfmerge['neo_reference_id'] = self.dfmerge['neo_reference_id'].astype(int)


        self.dfmerge['name'] = self.dfmerge['name'].str.replace('\'', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace('{', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace('}', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace(']', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace('[', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace('(', '')
        self.dfmerge['name'] = self.dfmerge['name'].str.replace(')', '')
        parts = self.dfmerge['name'].str.split(':')
        self.dfmerge['name'] = parts.str[1]


        self.dfmerge['name_limited'] = self.dfmerge['name']
        parts = self.dfmerge['name_limited'] = self.dfmerge['name_limited'].str.split(' ')
        self.dfmerge['name_limited'] = parts.str[2]
        self.dfmerge.insert(4, 'name_limited', self.dfmerge.pop('name_limited'))



        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace('\'', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace('{', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace('}', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace(']', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace('[', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace('(', '')
        self.dfmerge['designation'] = self.dfmerge['designation'].str.replace(')', '')
        parts = self.dfmerge['designation'].str.split(':')
        self.dfmerge['designation'] = parts.str[1]
        parts2 = self.dfmerge['designation'].str.split(' ')
        self.dfmerge['designation'] = parts2.str[1]



        self.dfmerge['nasa_jpl_url'] = self.dfmerge['nasa_jpl_url'].str.replace('\'', '')
        self.dfmerge['nasa_jpl_url'] = self.dfmerge['nasa_jpl_url'].str.replace('{', '')
        self.dfmerge['nasa_jpl_url'] = self.dfmerge['nasa_jpl_url'].str.replace('}', '')
        self.dfmerge['nasa_jpl_url'] = self.dfmerge['nasa_jpl_url'].str.replace(']', '')
        self.dfmerge['nasa_jpl_url'] = self.dfmerge['nasa_jpl_url'].str.replace('[', '')
        parts = self.dfmerge['nasa_jpl_url'].str.split(' ')
        self.dfmerge['nasa_jpl_url'] = parts.str[2]

        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace('\'', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace('{', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace('}', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace(']', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace('[', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace('(', '')
        self.dfmerge['absolute_magnitude_h'] = self.dfmerge['absolute_magnitude_h'].str.replace(')', '')
        parts = self.dfmerge['absolute_magnitude_h'].str.split(':')
        self.dfmerge['absolute_magnitude_h'] = parts.str[1]

        return self.dfmerge




    def process(self):
        self.dfmerge = self.table()
        return self.dfmerge


# Usage
df2 = pd.read_csv('/Volumes/Maestria/GitHub/NEO/NEO/API_test/df_2.csv')
df2 = Clean(df2).process()
# df2 = Append(df2).separation()
df2

JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 79 (char 78) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimiter: line 1 column 80 (char 79) - Check the JSON format.
JSON decode error: Expecting ',' delimit

,links,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,is_potentially_hazardous_asteroid,close_approach_data,orbit_data,sentry_object_data
0,http://api.nasa.gov/neo/rest/v1/neo/54296386?a...,54296386,54296386,2022 OC17,OC17,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.36,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1921-11-14', 'close_a...","[, 'orbital_data': {'orbit_id': '9', 'orbit_...",'is_sentry_object': False}
1,http://api.nasa.gov/neo/rest/v1/neo/54292695?a...,54292695,54292695,2022 PA,PA,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.06,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1983-08-19', 'close_a...","[, 'orbital_data': {'orbit_id': '2', 'orbit_...",'is_sentry_object': False}
2,http://api.nasa.gov/neo/rest/v1/neo/54292849?a...,54292849,54292849,2022 PB,PB,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.55,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1937-09-06', 'close_a...","[, 'orbital_data': {'orbit_id': '33', 'orbit...",'is_sentry_object': False}
3,http://api.nasa.gov/neo/rest/v1/neo/54292858?a...,54292858,54292858,2022 PC,PC,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.74,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1992-07-31', 'close_a...","[, 'orbital_data': {'orbit_id': '37', 'orbit...",'is_sentry_object': False}
4,http://api.nasa.gov/neo/rest/v1/neo/54292859?a...,54292859,54292859,2022 PD,PD,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.44,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1921-08-26', 'close_a...","[, 'orbital_data': {'orbit_id': '5', 'orbit_...",'is_sentry_object': False}
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6829,http://api.nasa.gov/neo/rest/v1/neo/54487064?a...,54487064,54487064,2024 TS1,TS1,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.659,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '1949-10-22', 'close_a...","[, 'orbital_data': {'orbit_id': '1', 'orbit_...",'is_sentry_object': False}
6830,http://api.nasa.gov/neo/rest/v1/neo/54487065?a...,54487065,54487065,2024 TT1,TT1,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.72,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '2024-11-21', 'close_a...","[, 'orbital_data': {'orbit_id': '1', 'orbit_...",'is_sentry_object': False}
6831,http://api.nasa.gov/neo/rest/v1/neo/54487066?a...,54487066,54487066,2024 TU1,TU1,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.081,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '2024-10-11', 'close_a...","[, 'orbital_data': {'orbit_id': '1', 'orbit_...",'is_sentry_object': True
6832,http://api.nasa.gov/neo/rest/v1/neo/54487067?a...,54487067,54487067,2024 TV1,TV1,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.676,{ 'estimated_diameter': {'kilometers': {'estim...,'is_potentially_hazardous_asteroid': False,"{'close_approach_date': '2024-11-25', 'close_a...","[, 'orbital_data': {'orbit_id': '1', 'orbit_...",'is_sentry_object': False}


In [21]:
import pandas as pd
import json
import ast

df = pd.read_csv('/Volumes/Maestria/GitHub/NEO/NEO/API_test/df_2.csv')

#2809
df2t = df.iloc[[2809]]  # Process each row
df2t = pd.DataFrame(df2t)
df2t['data'] = df2t['neo_data']
df2t = df2t.drop(columns=['neo_data'])
df2t = df2t.astype(str)


parts = df2t['data'].str.split(',')
parts1 = df2t['data'].str.split(',')


# t1 = ''
# for i in range(0,8):
#     t1 += data[i]
# parts2 = t1.str.split(':')
# t2 = parts2.str[6]
# parts3 = t2.str.split('\'')
# name = parts3.str[1]
# return name



specific = 'name_limited'
contains_string = parts.apply(lambda x: specific in x[4] if len(x) > 4 else False)

if contains_string.any():
    df2t['name_limited'] = parts.str[4]
    df2t['designation'] = parts.str[5]
    df2t['nasa_jpl_url'] = parts.str[6]
    df2t['absolute_magnitude_h'] = parts.str[7]
    df2t['estimated_diameter1'] = ' '
    for i in range(8,16):
        df2t['estimated_diameter1'] += parts.str[i] + ','


else:
    df2t['name_limited'] = ' '
    df2t['designation'] = parts.str[4]
    df2t['nasa_jpl_url'] = parts.str[5]
    df2t['absolute_magnitude_h'] = parts.str[6]
    df2t['estimated_diameter1'] = ' '
    for i in range(7,15):
        df2t['estimated_diameter1'] += parts.str[i] + ','
 


parts = df2t['estimated_diameter1'].str.split(' ')

df2t['estimated_diameter'] = ''
for i in range(3,23):
    df2t['estimated_diameter'] += parts.str[i] + ','

df2t.drop(columns=['estimated_diameter1'], inplace=True)


parts1 = df2t['data'].str.split('[')
df2t['approachdata'] = ' '
df2t['approachdata'] = parts1.str[1]
parts2 = df2t['approachdata'].str.split(']')
df2t['approach'] = ' '
df2t['approach'] = parts2.str[0]



# j = 0
# while j < 200:
#     print(f'part {j}: {parts5.str[j]} \n')
#     j += 1


df2t

,data,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,estimated_diameter,approachdata,approach
2809,{'links': {'self': 'http://api.nasa.gov/neo/re...,'name_limited': 'Miorita','designation': '622577','nasa_jpl_url': 'https://ssd.jpl.nasa.gov/too...,'absolute_magnitude_h': 19.71,"{'kilometers':,{'estimated_diameter_min':,0.30...","{'close_approach_date': '1902-11-11', 'close_a...","{'close_approach_date': '1902-11-11', 'close_a..."


In [22]:
class append:
    def __init__(self,df2):
        self.df = df2 

    def name(self, data):

        t1 = ''
        for i in range(0,8):
            t1 += data[i]
        parts2 = t1.str.split(':')
        t2 = parts2.str[6]
        parts3 = t2.str.split('\'')
        name = parts3.str[1]
        return name
    

    def designation(self, data):

        specific = 'name_limited'
        contains_string = self.parts1.apply(lambda x: specific in x[4] if len(x) > 4 else False)

        if contains_string.any():
            name_limited = data[4]
            designation = data[5]
            nasa_jpl_url = data[6]
            abs_magnitude_h = data[7]
            diameter = ' '
            for i in range(8,16):
                diameter += data[i] + ','
            hazard = data[16]
        else:
            name_limited = ' '
            designation = data[4]
            nasa_jpl_url = data[5]
            abs_magnitude_h = data[6]
            diameter = ' '
            for i in range(7,15):
                diameter += data[i] + ','
            hazard = data[15]


        return name_limited, designation, nasa_jpl_url, abs_magnitude_h, diameter, hazard
    
    def separation(self):

        # Assuming df2 is already defined
        df_result = pd.DataFrame()  # Initialize an empty DataFrame to store results

        for i in range(len(self.df)):
            df2t = df2.iloc[[i]]  # Process each row
            df2t = pd.DataFrame(df2t)
            df2t['data'] = df2t['neo_data']
            df2t = df2t.drop(columns=['neo_data'])
            df2t = df2t.astype(str)

            parts = df2t['data'].str.split(' ')
            parts1 = df2t['data'].str.split(',')
            self.parts1 = parts1
            name_limited, designation, nasa_jpl_url, abs_magnitude_h, diameter, hazard = self.designation(parts1.str)
            
            df2t['links'] = parts.str[1] + parts.str[2]
            df2t['id'] = parts.str[4]
            df2t['id'] = df2t['id'].str.replace('\'', '')
            df2t['id'] = df2t['id'].str.replace(',', '')
            df2t['id'] = df2t['id'].astype(int)


            df2t['neo_reference_id'] = parts.str[6]
            df2t['neo_reference_id'] = df2t['neo_reference_id'].str.replace('\'', '')
            df2t['neo_reference_id'] = df2t['neo_reference_id'].str.replace(',', '')
            df2t['neo_reference_id'] = df2t['neo_reference_id'].astype(int)


            df2t['name'] = self.name(parts1.str)


            df2t['name_limited'] = name_limited
            parts = df2t['name_limited'].str.split(' ')
            df2t['name_limited'] = parts.str[2]
            df2t['name_limited'] = df2t['name_limited'].astype(str)
            df2t['name_limited'] = df2t['name_limited'].str.replace('\'', '')


            df2t['designation'] = designation
            parts = df2t['designation'].str.split(' ')
            df2t['designation'] = parts.str[2]
            df2t['designation'] = df2t['designation'].astype(str)
            df2t['designation'] = df2t['designation'].str.replace('\'', '')


            df2t['nasa_jpl_url'] = nasa_jpl_url
            parts = df2t['nasa_jpl_url'].str.split(' ')
            df2t['nasa_jpl_url'] = parts.str[2]
            df2t['nasa_jpl_url'] = df2t['nasa_jpl_url'].astype(str)
            df2t['nasa_jpl_url'] = df2t['nasa_jpl_url'].str.replace('\'', '')


            df2t['absolute_magnitude_h'] = abs_magnitude_h
            parts = df2t['absolute_magnitude_h'].str.split(' ')
            df2t['absolute_magnitude_h'] = parts.str[2]
            df2t['absolute_magnitude_h'] = df2t['absolute_magnitude_h'].astype(str)
            df2t['absolute_magnitude_h'] = df2t['absolute_magnitude_h'].str.replace('\'', '')
            df2t['absolute_magnitude_h'] = df2t['absolute_magnitude_h'].astype(float)


            df2t['estiated_diameter1'] = diameter
            df2t['estiated_diameter1'] = df2t['estiated_diameter1'].str.rstrip(',')
            parts = df2t['estiated_diameter1'].str.split(' ')

            df2t['estiated_diameter'] = ' '
            for i in range(3,23):
                df2t['estiated_diameter'] += parts.str[i]
            df2t.drop(columns=['estiated_diameter1'], inplace=True)


            df2t['is_potentially_hazardous_asteroid'] = hazard
            parts = df2t['is_potentially_hazardous_asteroid'].str.split(' ')
            df2t['is_potentially_hazardous_asteroid'] = parts.str[2]
            df2t['is_potentially_hazardous_asteroid'] = df2t['is_potentially_hazardous_asteroid'].astype(str)
            df2t['is_potentially_hazardous_asteroid'] = df2t['is_potentially_hazardous_asteroid'].str.lower() == 'true'
            df2t['is_potentially_hazardous_asteroid'] = df2t['is_potentially_hazardous_asteroid'].astype(bool)

            parts1 = df2t['data'].str.split('[')
            df2t['approachdata'] = ' '
            df2t['approachdata'] = parts1.str[1]
            parts2 = df2t['approachdata'].str.split(']')
            df2t['close_approach_data'] = ' '
            df2t['close_approach_data'] = parts2.str[0]
            df2t.drop(columns=['approachdata'], inplace=True)

            
                        


            df_result = pd.concat([df_result, df2t], ignore_index=True)  # Append to the result DataFrame

        # df_result = df_result.drop(columns=['data'])
        df_result = df_result.drop(columns=['data'])
        self.df_result = df_result

        return self.df_result
    


df2 = pd.read_csv('/Volumes/Maestria/GitHub/NEO/NEO/API_test/df_2.csv')

df2 = append(df2).separation()
df2

,links,id,neo_reference_id,name,name_limited,designation,nasa_jpl_url,absolute_magnitude_h,estiated_diameter,is_potentially_hazardous_asteroid,close_approach_data
0,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54296386,54296386,(2022 OC17),nan,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.360,{'kilometers':{'estimated_diameter_min':0.142...,False,"{'close_approach_date': '1921-11-14', 'close_a..."
1,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54292695,54292695,(2022 PA),nan,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.060,{'kilometers':{'estimated_diameter_min':0.016...,False,"{'close_approach_date': '1983-08-19', 'close_a..."
2,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54292849,54292849,(2022 PB),nan,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,21.550,{'kilometers':{'estimated_diameter_min':0.130...,False,"{'close_approach_date': '1937-09-06', 'close_a..."
3,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54292858,54292858,(2022 PC),nan,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,23.740,{'kilometers':{'estimated_diameter_min':0.047...,False,"{'close_approach_date': '1992-07-31', 'close_a..."
4,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54292859,54292859,(2022 PD),nan,2022,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.440,{'kilometers':{'estimated_diameter_min':0.034...,False,"{'close_approach_date': '1921-08-26', 'close_a..."
...,...,...,...,...,...,...,...,...,...,...,...
6829,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54487064,54487064,(2024 TS1),nan,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,27.659,{'kilometers':{'estimated_diameter_min':0.007...,False,"{'close_approach_date': '1949-10-22', 'close_a..."
6830,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54487065,54487065,(2024 TT1),nan,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,22.720,{'kilometers':{'estimated_diameter_min':0.075...,False,"{'close_approach_date': '2024-11-21', 'close_a..."
6831,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54487066,54487066,(2024 TU1),nan,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,26.081,{'kilometers':{'estimated_diameter_min':0.016...,False,"{'close_approach_date': '2024-10-11', 'close_a..."
6832,{'self':'http://api.nasa.gov/neo/rest/v1/neo/5...,54487067,54487067,(2024 TV1),nan,2024,https://ssd.jpl.nasa.gov/tools/sbdb_lookup.htm...,24.676,{'kilometers':{'estimated_diameter_min':0.030...,False,"{'close_approach_date': '2024-11-25', 'close_a..."
